# The Hough Evaluation Platform
__Author__: David Northcote, Louise Crockett, Paul Murray, & Bob Stewart <br>
__Date__: 11/01/2022 <br> 
__Organisation__: The University of Strathclyde <br>
__Contact__: david.northcote@strath.ac.uk <br>

The contents of this notebook demonstrate the Hough Evaluation Platform (HEP), which is a useful tool to validate Field Programmable Gate Array (FPGA) architectures of the Line Hough Transform (LHT). The LHT is a robust and accurate algorithm for extracting lines from digital images. The LHT is highly computational and requires a dedicated FPGA processor to provide faster processing capabilities for embedded applications. The system in this notebook was created to extract lines from digital images of 1920x1080 pixels operating at a system clock frequency of 200 MHz.

<img src="./assets/system_overview.png" alt="System Overview" width="1024"/>

The HEP can calculate the time taken to apply the LHT to an image using a custom FPGA architecture and is capable of inspecting the Hough Parameter Space (HPS) using PYNQ's signal tracing and visualisation capbilities. The HEP can also calculate a new quantitative measurement for analysing the HPSs produced from different LHT architectures. This measurement is named the Peak-to-Mean Vote Ratio (PMVR) and is useful for quantifying the ability to reliably extract and separate peaks in the HPS from other neighbouring locations.

Initially, an edge detector is applied to a greyscale image using Sobel operators. The edge image is passed to the LHT architecture in the PL to obtain the Hough Parameter Space (HPS). The HPS can be optionally manipulated in Python for analysis or the resulting lines can be drawn to a blank array for inspection and superimposed on the original image.

The LHT hardware accelerator has been designed using primative blocks from Mathworks HDL Coder. The Xilinx Vivado design suite was used to integrate the hardware design and generate a bitstream for the FPGA configuration. This was achieved using a Mathworks' HDL Coder reference design to improve development time, allowing LHT architectures to be rapdily validated using the HEP.

## Initialise the Overlay
We will begin by download the HEP bitstream that consists of an LHT architecture that requires validation. A separate class has been created in ```hough_evaluation.py``` named _HoughEvaluation_ and can be initialised using the width and height of the input image. The clock frequency of the system is also required to calculate the processing time of the architecture. Running the code cell below will create the ```hep``` and also download the bitstream to the FPGA. A custom bitstream may be used instead by setting the ```bitfile_name``` argument to the bitstream's location.

In [ ]:
from hep import HoughEvaluation

hep = HoughEvaluation(width=1920, 
                      height=1080, 
                      frequency=200e6)

## Image Preprocessing
Image preprocessing is performed using the openCV library. The HoughEvaluation class has a set of premade functions that handle all of the image preprocessing required for the LHT.

Initially, an image is read into the workspace using the ```img = imread()``` function. When no image is supplied, _imread_ will use a default image provided in this package.

In [ ]:
img = hep.imread()
hep.display(img)

The image is then converted to greyscale so that edge detection operators can be applied. This is performed using the ```grey = rgb2grey(rgb)``` function. In this demonstration, the Sobel operators are used to find the edges of a greyscale image. This is easily achieved using ```edge = sobel_edge(grey, threshold)```, where _threshold_ is the largest magnitude value required to set a pixel to an edge.

In [ ]:
grey = hep.rgb2grey(img)
edge = hep.sobel_edge(grey, 100)
hep.display(edge)

## Evaluating an LHT Architecture
As mentioned previously, this bitstream has been configured with an instance of the HEP and an LHT architecture for evaluation. To perform the Hough Transform, two DMAs are require; one for the input image, and another to capture the output HPS. A Hough Performance Analyser (HPA) module is connected to the input and output of the LHT architecture (named _AXIS_Architecture_0_) so that it can perform timing analysis and calculate the PMVR. This can be seen in the Vivado IP diagram shown below.

In [ ]:
%timeit hps = hep.hough_lines(edge)

That was fast! We just performed the LHT on our candidate image where it was performed 10 times using the ```%timeit``` Jupyter magic command. You can see timing result above, however, this doesn't give an accurate reflection of the hardware processing time as it also takes into account transfers to/from system memory. To obtain an accurate representation of architecture processing time, the HPA module can be used instead. This has already obtained an accurate timing analysis of the LHT architecture and can be given by running the cell below.

In [ ]:
hep.time

The plotly.surface api can be used to analyse peaks that have accumulated in the HPS. The plot creates a 3-dimensional view of the HPS produced by the LHT architecture and can be created using ```f = plot(hps)```. It is possible to click and drag the plot to obtain different views of the resulting data. The highest peak in the HPS can be easily found by navigating around the plot (e.g. zooming, panning).

__Note: The plot will take a moment to initialise.__

In [ ]:
hps = hep.hough_lines(edge)
fsurface = hep.plot_surface(hps)
fsurface.write_image("plotly_hps_surface.png")

In [ ]:
fheatmap = hep.plot_heatmap(hps)
fheatmap.write_image("plotly_hps_heatmap.png")

Finally, it is also possible to obtain the PMVR of the HPS. The PMVR is the ratio of the highest peak in the HPS, to the mean votes in the nonzero locations of the HPS. It is effective in determining the ability to extract a peak from neighbouring locations in the HPS. The PMVR is given as follows.

In [ ]:
hep.get_pmvr(hps)